In [ ]:
from iterative_solvers import deflated_fgmres, deflated_fgmres3, deflated_pcg, deflated_gmres, recycled_fgmres, extend_orthonormal_basis, gmres_deflated_preconditioned, extend_A_orthonormal_basis
from operators import BlockMatrixOperator, BlockTriangularPreconditioner
from tunnel_with_subdomains import load_matrices_from_hdf5, load_vectors_from_hdf5

problem_no = 19
path_to_data = f"exported_data/testrun_{problem_no}/"

# loading
A_blocks = load_matrices_from_hdf5(f"{path_to_data}data_as_blocks.h5")
Ptdiagblocks = load_matrices_from_hdf5(f"{path_to_data}data_diag_preco.h5")
Ptblocks = load_matrices_from_hdf5(f"{path_to_data}data_triang_preco.h5")
rhs_u, rhs_p = load_vectors_from_hdf5(f"{path_to_data}data_rhs.h5")  # each rhs_* is a list of ndarrays
sol_u, sol_p = load_vectors_from_hdf5(f"{path_to_data}data_sol.h5")  # each rhs_* is a list of ndarrays

In [ ]:
import numpy as np
A_mat = BlockMatrixOperator(A_blocks)
# Pt_inv = BlockTriangularPreconditioner(Ptblocks, False)
Pt_inv = BlockTriangularPreconditioner(Ptdiagblocks, False)


In [ ]:
import numpy as np
from scipy.sparse.linalg import gmres, LinearOperator

iter_list = []
res_list = []
true_res_list = []
err_list = []
Z = None
Z_list = []
x_sol = None

warm_up = 10

for index in range(716):
    rhs_vec = np.concatenate([rhs_u[index], rhs_p[index]])
    sol_vec = np.concatenate([sol_u[index], sol_p[index]])
    if index < warm_up:
        x_sol, iters, res_hist = deflated_fgmres3(A_mat, rhs_vec, Pt_inv, maxits=30, tol=1e-10, x0=x_sol)
    else:
        x_sol, iters, res_hist = deflated_fgmres3(A_mat, rhs_vec, Pt_inv, maxits=30, tol=1e-10, Z=Z)

    if np.linalg.norm(x_sol) > 10:
        Z_list.append(x_sol)
    if index >= (warm_up - 1):
        Z = np.column_stack(Z_list)
        Z, _ = np.linalg.qr(Z, mode='reduced')

    # if Z is None:
    #     if np.linalg.norm(x_sol) > 0:
    #         Z = x_sol / np.linalg.norm(x_sol)
    #         Z = Z.reshape(-1, 1)
    # else:
    #     Z = extend_A_orthonormal_basis(A_mat, Z, x_sol.reshape(-1, 1), droptol=1e-6, max_size=1000, verbose=True)
    #     if index % 100 == warm_up:
    #         # Z, _ = np.linalg.qr(Z)
    #         pass

    iter_list.append(iters)
    res_list.append(np.linalg.norm(A_mat(x_sol) - rhs_vec)/np.linalg.norm(rhs_vec))
    true_res_list.append(np.linalg.norm(A_mat(sol_vec) - rhs_vec)/np.linalg.norm(rhs_vec))
    err_list.append(np.linalg.norm(x_sol - sol_vec)/np.linalg.norm(sol_vec))
    print(
        f"Inex {index}: {iters}, {
            np.linalg.norm(
                Pt_inv(A_mat(x_sol) -
                rhs_vec)) /
            np.linalg.norm(rhs_vec)}, {
            np.linalg.norm(
                x_sol -
                sol_vec) /
            np.linalg.norm(sol_vec)} {
            np.linalg.norm(
                Pt_inv(A_mat(sol_vec) -
                rhs_vec)) /
            np.linalg.norm(rhs_vec)}")

print(f"Number of iterations: {iters}")
print(f"Relative residual: {res_hist[-1]}")
print(f"Relative error: {np.linalg.norm(x_sol - sol_vec)/np.linalg.norm(sol_vec)}")

In [ ]:
print(
    f"Inex {index}: {iters}, {
        np.linalg.norm(
            A_mat(x_sol) -
            rhs_vec) /
        np.linalg.norm(rhs_vec)}, {
        np.linalg.norm(
            x_sol -
            sol_vec) /
        np.linalg.norm(sol_vec)} {
        np.linalg.norm(
            A_mat(sol_vec) -
            rhs_vec) /
        np.linalg.norm(rhs_vec)}")
print(
    f"{
        np.linalg.norm(
            A_mat(x_sol - sol_vec)) /
        np.linalg.norm(
            x_sol -
            sol_vec)}")
plt.plot(x_sol -
         sol_vec)


In [ ]:
import matplotlib.pyplot as plt
plt.imshow(Z.T @ Z)
plt.colorbar()

In [ ]:
import matplotlib.pyplot as plt
plt.semilogy(res_hist)

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(iter_list)
plt.show()

plt.figure()
plt.semilogy(res_list)
plt.show()

plt.figure()
plt.semilogy(err_list)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(iter_list)
plt.show()

plt.figure()
plt.semilogy(res_list)
plt.semilogy(true_res_list)
plt.show()

plt.figure()
plt.semilogy(err_list)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(iter_list)
plt.show()

plt.figure()
plt.semilogy(res_list)
plt.semilogy(true_res_list)
plt.show()

plt.figure()
plt.semilogy(err_list)
plt.show()

In [ ]:
import numpy as np
from scipy.sparse.linalg import gmres, LinearOperator

iter_list = []
res_list = []
true_res_list = []
err_list = []
Z = None
Z_list = []
x_sol = None

warm_up = 716

for index in range(716):
    rhs_vec = np.concatenate([rhs_u[index], rhs_p[index]])
    sol_vec = np.concatenate([sol_u[index], sol_p[index]])
    if index < warm_up:
        x_sol, iters, res_hist = deflated_fgmres(A_mat, rhs_vec, Pt_inv, maxits=30, tol=1e-14, x0 = x_sol)
    else:
        x_sol, iters, res_hist = deflated_fgmres(A_mat, rhs_vec, Pt_inv, maxits=30, tol=1e-14, Z=Z)

    # Z_list.append(x_sol)
    # if index >= (warm_up - 1):
    #     Z = np.column_stack(Z_list[index - (warm_up - 1):])
    #     Z, _ = np.linalg.qr(Z)

    # if Z is None:
    #     if np.linalg.norm(x_sol) > 0:
    #         Z = x_sol / np.linalg.norm(x_sol)
    #         Z = Z.reshape(-1, 1)
    # else:
    #     Z = extend_orthonormal_basis(Z, x_sol.reshape(-1,1), droptol=1e-6, max_size=100, verbose=True)
    #     if index % 100 == 15:
    #         Z, _ = np.linalg.qr(Z)

    iter_list.append(iters)
    res_list.append(np.linalg.norm(A_mat(x_sol) - rhs_vec)/np.linalg.norm(rhs_vec))
    true_res_list.append(np.linalg.norm(A_mat(sol_vec) - rhs_vec)/np.linalg.norm(rhs_vec))
    err_list.append(np.linalg.norm(x_sol - sol_vec)/np.linalg.norm(sol_vec))
    print(f"Inex {index}: {iters}, {np.linalg.norm(A_mat(x_sol) - rhs_vec)/np.linalg.norm(rhs_vec)}, {np.linalg.norm(x_sol - sol_vec)/np.linalg.norm(sol_vec)} {np.linalg.norm(A_mat(sol_vec) - rhs_vec)/np.linalg.norm(rhs_vec)}")

print(f"Number of iterations: {iters}")
print(f"Relative residual: {res_hist[-1]}")
print(f"Relative error: {np.linalg.norm(x_sol - sol_vec)/np.linalg.norm(sol_vec)}")

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(iter_list)
plt.show()

plt.figure()
plt.semilogy(res_list)
plt.semilogy(true_res_list)
plt.show()

plt.figure()
plt.semilogy(err_list)
plt.show()